Kaito Minami, Nick Tarallo

CS 4973 Math Word Problems

# Math Word Problems
In this project, we will use an LLM to solve math word problems, such as this one 1:

Very early this morning, Elise left home in a cab headed for the hospital. Fortunately, the roads were clear, and the cab company only charged her a base price of $3, and $4 for every mile she traveled. If Elise paid a total of $23, how far is the hospital from her house?

We will explore several prompting strategies, some of which will be more effective than the others at solving math word problems. However, all the strategies are quite generic and will be broadly applicable to a wide variety of tasks. We will also use this project to introduce the OpenAI Completions API, which is a widely used API for LLMs.

### Prerequisites
You will need to comfortable with text processing and regular expressions. If you are not, we recommend the following resources:

1. Chapter 2 of Speech and Language Processing by Dan Jurafsky and James H. Martin is an in-depth introduction to regular expressions.

2. The Regular Expression HOWTO by A.M. Kuchling is a gentler introduction to regular expressions in Python.

## The Completions API
The LLM that we will use in this assignment is Meta Llama 3.1 (8B). (We are deliberately not using Llama 3.1 Instruct, which is the instruction-tuned or “chat model”. The instruction-tuned model is even more capable on math word problems. However, the techniques that we will explore are also useful when working with instruction-tuned model to solve harder problems than math word problems.) Although it is a relatively small LLM, it is very capable for its size. The following code shows you how to query it:

In [1]:
from openai import OpenAI

URL = "http://199.94.61.113:8000/v1/"
KEY = "tarallo.n@northeastern.edu:OQEgetkQ6LBofgWW4jsC"

client = OpenAI(base_url=URL, api_key=KEY)

resp = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B",
    temperature=0.2,
    max_tokens=100,
    stop=["\n"],
    prompt="Shakespeare was"

)
print(resp.choices[0].text)


 a genius. He was a genius who was able to take the most complex of subjects and make them understandable to the common man. He was a genius who was able to take the most complex of subjects and make them understandable to the common man. He was a genius who was able to take the most complex of subjects and make them understandable to the common man. He was a genius who was able to take the most complex of subjects and make them understandable to the common man. He was a genius who


When you run this code, you may see something like this:

born in Stratford-upon-Avon, England, in 1564. His father was a glove-maker and his mother was the daughter of a landowner. Shakespeare attended grammar school in Stratford, and his education was broad and rigorous. He was well versed in literature, history, and the classics. At the age of 18, he married Anne Hathaway, who was eight years his senior. The couple had three children: Susanna, Hamnet, and Judith. Shakespeare began

You should play around with the code above and review the Completions API Reference and the Completions Guide. Note that the model that we host does not support every optional argument that the API documents. However, you can set the stop sequences, set the sampling temperature, setup nucleus sampling, and control the generation length.

## Zero-Shot Prompting
A base model is not specifically designed to answer questions. All it does is complete the prompt with likely text. For example, if we prompt the model with exactly the text of the math word problem above, you may get an answer, an explanation, or even a continuation of the problem. For example, after five attempts at temperature 0.2, I got the model to produce the following hint instead of the answer:

Hint: You can use the equation 3 + 4x = 23, where x is the distance in miles.

**Task 1:** Your first task is to figure out how to prompt the model so that it fairly reliably produces an answer for a math word problem: an answer that is always a number. To do so, write a pair of functions that (1) take a math word problem and turns it into a prompt that elicits a direct answer from the LLM and (2) takes the LLM response, which will always be a string, and turns it into a number. The latter function should return None if the LLM does not produce a number as directed.

In [55]:
from typing import List, Optional

def prompt_zero_shot(problem: str) -> str:
    return problem + "\n\nAnswer without units ="

def extract_zero_shot(completion: str) -> Optional[int]:
    try:
        return int(completion.strip())
    except:
        print(completion)
        return None
    


The two functions above should not use the Completions API. Instead, put them together using the following code:

In [62]:
def solve_zero_shot(problem: str) -> Optional[int]:
    resp = client.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B",
        temperature=0.2,
        prompt=prompt_zero_shot(problem),
        max_tokens=2
    )
    return extract_zero_shot(resp.choices[0].text)



**Task 2:** For this task, you will work with a list of math word problems and their answers. For example:

In [37]:
EASY = [
    { 
        "question": "I ate 2 pears and 1 apple. How many fruit did I eat?",
        "answer": 3 
    },
    {
        "question": "I had ten chocolates but ate one. How many remain?",
        "answer": 9
    }
]


Your function should take a list of problems, such as the one above, and compute the accuracy of the LLM on that list:

In [64]:
from tqdm.auto import tqdm

def accuracy_zero_shot(problems: List[dict]) -> float:
    num_correct = 0
    for problem in tqdm(problems):
        for i in range(5):
            result = solve_zero_shot(problem['question'])
            if result == problem['answer']:
                num_correct += 1
            else:
                print(f'Expected: {problem["answer"]}')
                print(f'Got: {result}')

    return num_correct / (5 * len(problems))

accuracy_zero_shot(EASY)


  0%|          | 0/2 [00:00<?, ?it/s]

1.0

Your code must not raise an exception, no matter what the LLM returns. So, make sure you handle any exceptions raised by solve_zero_shot.

**Task 3:** The dataset nuprl/llm-systems-math-word-problems has 50 math word problems in its test set, and you can load it as follows:

In [65]:
import datasets

TEST = datasets.load_dataset("nuprl/llm-systems-math-word-problems", split="test")
print(TEST)

print(accuracy_zero_shot(TEST))


Found cached dataset parquet (C:/Users/gardy/.cache/huggingface/datasets/nuprl___parquet/nuprl--llm-systems-math-word-problems-8d0e17f22584e8ca/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['question', 'answer'],
    num_rows: 50
})


  0%|          | 0/50 [00:00<?, ?it/s]

Expected: 18
Got: 8
Expected: 18
Got: 8
Expected: 18
Got: 8
Expected: 18
Got: 8
Expected: 18
Got: 8
  
Expected: 540
Got: None
Expected: 540
Got: 180
Expected: 540
Got: 180
Expected: 540
Got: 180
Expected: 20
Got: 40
Expected: 20
Got: 40
  
Expected: 20
Got: None
  
Expected: 20
Got: None
Expected: 260
Got: 140
Expected: 260
Got: 90
Expected: 260
Got: 120
Expected: 260
Got: 120
Expected: 260
Got: 100
Expected: 460
Got: 450
Expected: 460
Got: 450
Expected: 460
Got: 450
Expected: 460
Got: 450
 $450
Expected: 460
Got: None
Expected: 366
Got: 180
  
Expected: 366
Got: None
  
Expected: 366
Got: None
  
Expected: 366
Got: None
  
Expected: 366
Got: None
 $ 
Expected: 694
Got: None
Expected: 694
Got: 3
 $ 
Expected: 694
Got: None
Expected: 694
Got: 3
Expected: 694
Got: 3
Expected: 60
Got: 50
Expected: 60
Got: 20
Expected: 60
Got: 50
Expected: 60
Got: 25
Expected: 60
Got: 40
Expected: 230
Got: 2
 $ 
Expected: 57500
Got: None
 $ 
Expected: 57500
Got: None
 $ 
Expected: 57500
Got: None
 $ 
Expe

What accuracy do you get? Try re-running accuracy_zero_shot a few times. If you’re sampling with temperature, you will see that the result can vary significantly on each run. To get a stable result, update accuracy_zero_shot to try each problem n=5 times and report mean accuracy for each attempt.

For full credit, you need to get at least 10% accuracy and a stable result. If not, you can try to improve accuracy in a few ways:

1. You can try to improve the prompt in prompt_zero_shot. But, your prompt must still elicit a direct answer and should not given examples. (In the next part, we will use more sophisticated prompting techniques to elicit more complex responses.)

2. You may find that extract_zero_shot fails when the LLM produces strings such as "$23" or "1,200" Feel free to address these.

3. Experiment with different generation hyperparameters, such as temperature.

Answer: When running each problem 5 times, we get 0.128 accuracy.

### Tracking Progress
When you have a long list of problems, you will find it helpful to track progress. You could print after each problem, but that will fill up your screen quickly. Alternatively, use the tqdm library to display a compact progress bar.

## Few-Shot Prompting
With a zero-shot prompt, we are giving the model very limited guidance on what kind of answer we want. In fact, your zero-shot prompt was unlikely to be 100% reliable. There were probably a few problems where it did not produce a number. To address this, we’ll now explore few-shot prompting.2

**Task 4:** Implement the following functions:

In [66]:
def prompt_few_shot(problem: str) -> str:
    EXAMPLES = """
        John had 9 chocolate bars and ate 3 of them. How many does he have left?

        Answer without units = 6



        Alan has 4 cats and 3 dogs. How many total pets does he have?
        
        Answer without units = 7
    
        

        Katie has five pencils. Rebecca has three times as many. How many pencils does Rebecca have?
        
        Answer without units = 15



    """
    PROMPT = EXAMPLES + problem + '\n\nAnswer without units ='
    return PROMPT
    

def extract_few_shot(completion: str) -> Optional[int]:
    try:
        return int(completion.strip())
    except:
        print(completion)
        return None

def solve_few_shot(problem: str) -> Optional[int]:
    resp = client.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B",
        temperature=0.2,
        prompt=prompt_few_shot(problem),
        max_tokens=2
    )
    return extract_few_shot(resp.choices[0].text)

def accuracy_few_shot(problems: List[dict]) -> float:    
    num_correct = 0
    for problem in tqdm(problems):
        for i in range(5):
            result = solve_few_shot(problem['question'])
            if result == problem['answer']:
                num_correct += 1
            else:
                print(f'Expected: {problem["answer"]}')
                print(f'Got: {result}')

    return num_correct / (5 * len(problems))

accuracy_few_shot(EASY)


  0%|          | 0/2 [00:00<?, ?it/s]

1.0

The prompt that you construct should have a few example problems and answers. With your few-shot prompt, you should always get a numeric answer. However, we do not expect the accuracy to increase by very much over the zero-shot version.

In [67]:
TEST_FEW_SHOT = datasets.load_dataset("nuprl/llm-systems-math-word-problems", split="test")

print(accuracy_few_shot(TEST_FEW_SHOT))

Found cached dataset parquet (C:/Users/gardy/.cache/huggingface/datasets/nuprl___parquet/nuprl--llm-systems-math-word-problems-8d0e17f22584e8ca/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/50 [00:00<?, ?it/s]

 $20
Expected: 18
Got: None
 $20
Expected: 18
Got: None
Expected: 18
Got: 56
Expected: 18
Got: 56
Expected: 18
Got: 28
Expected: 3
Got: 4
Expected: 3
Got: 4
Expected: 3
Got: 4
Expected: 540
Got: 180
Expected: 540
Got: 180
Expected: 20
Got: 60
Expected: 20
Got: 40
Expected: 20
Got: 10
Expected: 20
Got: 10
Expected: 20
Got: 30
Expected: 260
Got: 90
Expected: 260
Got: 90
Expected: 260
Got: 100
Expected: 260
Got: 100
Expected: 260
Got: 120
Expected: 460
Got: 450
Expected: 460
Got: 450
Expected: 460
Got: 450
 $450
Expected: 460
Got: None
Expected: 460
Got: 450
Expected: 366
Got: 150
Expected: 366
Got: 150
Expected: 366
Got: 120
Expected: 366
Got: 150
Expected: 366
Got: 180
 $403
Expected: 694
Got: None
 $403
Expected: 694
Got: None
 $323
Expected: 694
Got: None
 $363
Expected: 694
Got: None
 $303
Expected: 694
Got: None
Expected: 60
Got: 55
Expected: 60
Got: 55
Expected: 60
Got: 55
Expected: 60
Got: 55
Expected: 60
Got: 55
 $42
Expected: 57500
Got: None
 $42
Expected: 57500
Got: None
 $42
E

## Chain-of-Thought Prompting
**For the rest of this assignment, feel free to use Copilot or other kinds of GenerativeAI.**

We’ll now explore chain-of-thought (COT) prompting, and see that it significantly increases accuracy on the task.3 For example, consider the following problem:

Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have?

Here is one way to reason through the answer:

7 pizzas are cut into 8 slices each. Thus the total number of slices is 7 * 8 = 56. Henry and 3 friends want to share the pizza equally, so the slices are divided between 4 people. Each person gets 56 / 4 = 14 slices.

**Task 5:** Your task is the implement the following functions:

In [71]:
def prompt_cot(problem: str) -> str:
    EXAMPLES = """Input: Sarah has 15 candies. She wants to share them equally between herself and 4 friends. How many candies will each person get?
 
    Reasoning: Sarah has 15 candies, and she needs to divide them equally among herself and 4 friends, which makes 5 people in total. To find out how many candies each person gets, we divide the total number of candies by the number of people. 15 ÷ 5 = 3 candies per person. So, each person gets 3 candies.

    Answer: 3

    Done
    

    Input: A bus can seat 40 people. If 5 buses are filled with passengers and there are 10 more people waiting to board, how many people in total need transportation?

    Reasoning: Each bus can hold 40 people. There are 5 buses, so the total number of people already seated in the buses is 5 * 40 = 200 people. Now, there are 10 more people waiting, so the total number of people that need transportation is the sum of those already seated and those waiting. Therefore, there are 200 + 10 = 210 people in total.

    Answer: 210

    Done


    Input: A library has 6 shelves. Each shelf holds 24 books. If the librarian adds 12 more books to one shelf, how many books are there in total now?
  
    Reasoning: The library initially has 6 shelves with 24 books on each shelf. The total number of books in the library before any additions is 6 * 24 = 144 books. Now, the librarian adds 12 more books to one shelf. So, the new total number of books becomes 144 + 12 = 156 books.

    Answer: 156

    Done

    

    """
    PROMPT = EXAMPLES + "Input: " + problem + "\n\nReasoning:"
    return PROMPT


def extract_cot(completion: str) -> Optional[int]:
    items = completion.split("Answer: ")
    if len(items) < 2:
        return None
    try:
        return int(items[1].strip())
    except:
        return None

def solve_cot(problem: str) -> Optional[int]:
    resp = client.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B",
        temperature=0.2,
        prompt=prompt_cot(problem),
        max_tokens=200,
        stop=['Done'],
    )
    return extract_cot(resp.choices[0].text)

def accuracy_cot(problems: List[dict]) -> float:    
    num_correct = 0
    for problem in tqdm(problems):
        for i in range(5):
            result = solve_cot(problem['question'])
            if result == problem['answer']:
                num_correct += 1
            else:
                print(f'Expected: {problem["answer"]}')
                print(f'Got: {result}')

    return num_correct / (5 * len(problems))


Start by writing the prompting function, which should prefix the problem with 2-3 COT examples. You’ll need to write the “thoughts” yourself. Do not use the problems from test split for the COT examples. Instead, you may use the problems from train split, or construct your own.

Given that your prompt elicits “thoughts” from the model, you will need to carefully extract the final answer in extract_cot. This will require more work than what you did in the earlier approaches. For full credit, you need to get at least 45% accuracy and a stable result.

In [72]:
TEST_COT = datasets.load_dataset("nuprl/llm-systems-math-word-problems", split="test")

print(accuracy_cot(TEST_COT))

Found cached dataset parquet (C:/Users/gardy/.cache/huggingface/datasets/nuprl___parquet/nuprl--llm-systems-math-word-problems-8d0e17f22584e8ca/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/50 [00:00<?, ?it/s]

Expected: 18
Got: None
Expected: 18
Got: None
Expected: 18
Got: None
Expected: 20
Got: 2
Expected: 20
Got: 40
Expected: 20
Got: 2
Expected: 260
Got: 12
Expected: 260
Got: None
Expected: 460
Got: None
Expected: 460
Got: None
Expected: 460
Got: None
Expected: 460
Got: None
Expected: 460
Got: None
Expected: 694
Got: None
Expected: 694
Got: None
Expected: 694
Got: None
Expected: 694
Got: None
Expected: 60
Got: None
Expected: 60
Got: None
Expected: 60
Got: None
Expected: 60
Got: None
Expected: 60
Got: None
Expected: 230
Got: None
Expected: 57500
Got: None
Expected: 57500
Got: None
Expected: 57500
Got: None
Expected: 57500
Got: None
Expected: 57500
Got: None
Expected: 26
Got: None
Expected: 26
Got: None
Expected: 26
Got: None
Expected: 26
Got: None
Expected: 26
Got: None
Expected: 2
Got: 5
Expected: 2
Got: 3
Expected: 243
Got: None
Expected: 243
Got: None
Expected: 243
Got: None
Expected: 16
Got: None
Expected: 16
Got: None
Expected: 16
Got: None
Expected: 16
Got: 480
Expected: 16
Got: None


## Program-Aided Language Models
For the final part, we will explore program-aided language models (PAL).4 PAL is a variation of chain-of-thought, but instead of prompting the model to reason in natural language, we prompt the model to produce a program that solves the problem, and then run that program.

For example, consider the example problem in the COT part. Instead of producing natural language, we could instead produce the following program:

In [ ]:
num_pizzas = 7
slices_per_pizza = 8
total_slices = num_pizzas * slices_per_pizza
num_people = 1 + 3
slices_per_person = total_slices / num_people


When we run this program, the value of slices_per_person will be the answer.

Dynamically running code: We strongly recommend encoding each program as a function that returns the answer. If you have a string that represents a function, you can define it with the builtin exec and get its result with the builtin eval. For example:

In [ ]:
CODE = """
def my_func():
    return 1 + 1
""".strip()

exec(CODE)
x = eval("my_func()")
assert x == 2

# If you know the name of the function, you can 
# call it without eval.
y = my_func()
assert y == 2


**Task 6:** Your task is to implement PAL with the following functions:

In [87]:
def prompt_pal(problem: str) -> str:
    return f"""# Question: Sarah has 15 candies. She wants to share them equally between herself and 4 friends. How many candies will each person get?

def question():
    # Sarah has 15 candies
    total_candies = 15
    # The total number of people is herself plus four of her friends
    num_people = 1 + 4
    # The amount of candies each person, which is the answer, is 3
    candies_per_person = total_candies // num_people
    return candies_per_person



# Question: A bus can seat 40 people. If 5 buses are filled with passengers and there are 10 more people waiting to board, how many people in total need transportation?

def question():
    # Each bus can hold 40 people
    bus_capacity = 40
    # There are 5 buses
    num_buses = 5
    # The amount of passengers already seated is
    passengers_on_buses = num_buses * bus_capacity
    # There are 10 people waiting to board
    waiting_people = 10
    # The total amount of people, which is the answer, is 210
    total_people = passengers_on_buses + waiting_people
    return total_people



# Question: A library has 6 shelves. Each shelf holds 24 books. If the librarian adds 12 more books to one shelf, how many books are there in total now?

def question():
    # The library initially has 6 shelves
    shelves = 6
    # The library initially has 24 books on each shelf
    books_per_shelf = 24
    # The total number of books on shelves is initially
    total_books = shelves * books_per_shelf
    # 12 books were added to a self
    added_books = 12
    # The total number of books on shelves at the end, which is the answer, is 156
    total_books_now = total_books + added_books
    return total_books_now 


# Question: {problem}

def question():"""

def extract_pal(completion: str) -> Optional[int]:
    CODE = "def question():\n    " + completion.strip()
    try:
        # print(CODE)
        exec(CODE)
        result = eval("question()")
        return int(result)
    except:
        print(CODE)
        return None

def solve_pal(problem: str) -> Optional[int]:
    resp = client.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B",
        temperature=0.2,
        prompt=prompt_pal(problem),
        max_tokens=300,
        stop=['# Question:'],
    )
    return extract_pal(resp.choices[0].text)
    

def accuracy_pal(problems: List[dict]) -> float:    
    num_correct = 0
    for problem in tqdm(problems):
        for i in range(5):
            result = solve_pal(problem['question'])
            if result == problem['answer']:
                num_correct += 1
            else:
                print(f'Expected: {problem["answer"]}')
                print(f'Got: {result}')

    return num_correct / (5 * len(problems))

print(accuracy_pal(TEST))

  0%|          | 0/50 [00:00<?, ?it/s]

Expected: 540
Got: 1260
Expected: 540
Got: 180
Expected: 20
Got: 2
Expected: 20
Got: 100
Expected: 20
Got: 40
Expected: 20
Got: 2
Expected: 20
Got: 2
def question():
    # The distance traveled by the first train is 80 miles
    train1_distance = 80
    # The distance traveled by the second train is 80 miles
    train2_distance = 80
    # The distance traveled by the first train the next day is 150 miles
    train1_distance_next_day = 150
    # The distance traveled by the second train the next day is 150 miles
    train2_distance_next_day = 150
    # The total distance traveled by the first train, which is the answer, is 230 miles
    train1_total_distance = train1_distance + train1_distance_next_day
    # The total distance traveled by the second train, which is the answer, is 230 miles
    train2_total_distance = train2_distance + train2_distance_next_day
    return train1_total_distance, train2_total_distance
Expected: 230
Got: None
Expected: 57500
Got: 125000
Expected: 57500
Got: 

For full credit, you need to get at least 60% accuracy and a stable result.

## What to Submit
You should submit two files. First, a Python file that implements all the functions above. Running this file should have no side-effects. Second, a Jupyter notebook that shows how you tested the work. This notebook should load the datasets and use the functions implemented in the first file. Make sure you save the cells’ outputs.

1. The word problems in this assignment are from the GSM8K (grade school mathematics) benchmark (Cobbe et al., 2021). ↩

2. The effectiveness of few-shot prompting was a key capability that distinguished GPT3 from GPT2 (Brown et al., 2020). ↩

3. Wei et al., 2022 introduced chain of thought prompting. ↩

4. Gao et al., 2023introduced program-aided language models. ↩

In [84]:
TEST[3]

{'question': "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?",
 'answer': 20}